In [ ]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276
#3) https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?Expires=1585601065&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=Lnpf6wT8rkozSh9av7U9nGuC7WAH6KuI2Cj3Y7G366gkGlh8D-Ie1Kc~TyBAUu~uMsVltleJcSv3p6TCm6HdFnhpyoTgLcYh6eFfvQwIUqbk1Bf4JZldgB~BDKUOwY1G0pA-HoKjvIAu3avO98SMO35upakm9OEBByd4nC9aXsjKRThd6bTpq1qIuuD9gh1l5FaM6hNRB0c2lCf4Q3adx7C3FW0NMwdWhcuF45A9f~dO3zTWWSQamoo5Otc-PHMMt96TetNcML~jy9ghgJeCPY6DJLUIwQAt03fENBluS~TjTJ17WD~n51xiRofb94fEJHoRHh0d-430LLwr7BX4IA__

In [ ]:
import numpy as np
import os
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
from pydub.playback import play
import matplotlib.pyplot as plt
#import seaborn as sn
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import kurtosis, skew
from scipy.signal import lfilter
import librosa
#import pysptk
import math
import sys

#settings
import config

## Reading recordings

In [ ]:
all_s=[]
all_label=[]
all_id=[]
all_fs=[]

In [ ]:
#Read wav data set

if config.featExtr_skip is False:
    print("Readings wavs...")

    #only list files in FOLDER_PATH directory
    wav_files = [f for f in os.listdir(config.FOLDER_PATH) if os.path.isfile(os.path.join(config.FOLDER_PATH, f))]
    for file_name in wav_files:
    
        fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
        #full path file name
        full_fname = config.FOLDER_PATH+file_name
        #print(full_fname)
    
        # load audio
        s = AudioSegment.from_wav(full_fname)
        #print(full_fname)
        all_s.append(s)
        #sampling rate:
        info = mediainfo(full_fname)
        fs = float(info['sample_rate'])
        all_fs.append(fs)
    
        #get ID of recording
        ID = fname_noExt.split('-')[-2] #for the current type of naming
        #print(file_name)
        #print(ID)
        all_id.append(ID)
    
        #get label
        label = fname_noExt.split('-')[-1] #for the current type of naming
        #print(label)
        all_label.append(label)

Listening to some of the audios

In [ ]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Dry')

In [ ]:
if config.featExtr_skip is False:
    np.where(np.array(all_label)=='Wet')

In [ ]:
if config.featExtr_skip is False:
    s=all_s[15]
    s

## Feature extraction

In [ ]:
import featureExtractionFunctions as featExtrLib

if config.featExtr_skip is False:

    feats = featExtrLib.feature_extraction_Step(all_s,all_id,all_label)
    
       

## Load  (or store) features 

In [ ]:
feats_fname = 'feats_df.pkl'

if config.featExtr_skip is False:
    #Store feature df
    feats.to_pickle(feats_fname)
else:
    #Load feature df
    feats = pd.read_pickle(feats_fname)

In [131]:
 feats = pd.read_pickle(feats_fname)

## Pre-processing of features

In [132]:
feats2 = featExtrLib.processingNaNvalues(feats)

In [133]:
label_dict = featExtrLib.createLabelDict_addLabel2df(feats2)
mean_std_feats = featExtrLib.frame_mean_std_chunk_modeling (feats2,label_dict)

## Model training

In [265]:
from sklearn.preprocessing import StandardScaler
X_train = mean_std_feats.iloc[:,8:21]
y_train =  mean_std_feats['label'].copy()

In [ ]:

ID_train = mean_std_feats['Id']
ID_list = ID_train.drop_duplicates().tolist()

In [ ]:
ID_train = mean_std_feats['Id']
ID_list = ID_train.drop_duplicates().tolist()

print(len(ID_list))

In [268]:
#Model training using Neural network

In [ ]:


ID_train = mean_std_feats['Id']
ID_list = ID_train.drop_duplicates().tolist()

ID_train.size
#ID_list.size
X_train.head(1)

for i in range(0,len(ID_list)):

        idnow = ID_list[i]
        ID_train_list = ID_train.to_list()
        val_index = [i for i, x in enumerate(ID_train_list) if x == idnow]
        train_index = [i for i, x in enumerate(ID_train_list) if x != idnow]

        X_train1, X_val1 = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train1, y_val1 = y_train.iloc[train_index], y_train.iloc[val_index]
   

        #normalize train set
        scaler = StandardScaler()
        scaler.fit(X_train1)
        X_trainNorm1 = scaler.transform(X_train1.values)
        X_valNorm1 = scaler.transform(X_val1.values)
        

In [275]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
from sklearn.model_selection import train_test_split
import keras.initializers as init 

X = X_trainNorm1
X.shape
number=LabelEncoder()
y = number.fit_transform(y_train1)
x_train = X
y_train = y
x_test = X_valNorm1 
y_test = number.fit_transform(y_val1)
print(y.shape)
print(X.shape)

num_labels = 2
filter_size = 2

def build_model_graph():
    # define the keras model
    model = Sequential()
    model.add(Dense(4, input_dim=13, activation='relu'))
    #model.add(Dense(1, activation='relu', kernel_initializer=init.glorot_uniform(seed=66)))
    model.add(Dense(1, activation='relu', kernel_initializer=init.VarianceScaling(scale=1.0, mode='fan_in', distribution='normal', seed=None)))
    #keras.initializers.Zeros()
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam')
   # model.add(Dense(12, input_dim=13, activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    # Compile the model
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

model = build_model_graph()


# Display model architecture summary 
#model.summary()

#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state = 127)

#print(x_train[1])

#print(x_test[1])

#print(y_train[1:5])

#print(y_test[1:5])


(1325,)
(1325, 13)


In [ ]:
# Get mean accuracy
accuracy=[]
for i in range(0,10):
    model = build_model_graph()
    num_epochs = 115
    num_batch_size = 30
    model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=0)
    score = model.evaluate(x_test, y_test, verbose=0) 
    accuracy.append(score[1])
    print("Training Accuracy: {0:.2%}".format(score[1]))

mean_accuracy = np.mean(accuracy)

print("Accuracy: {0:.2%}".format(mean_accuracy))   

In [ ]:
import matplotlib.pyplot as pyplot
pyplot.plot(hist.history['loss'], label='train')
pyplot.plot(hist.history['val_loss'], label='test')
pyplot.show()
